## Reproducible for the paper

# Radius of Information for Two Intersected Centered Hyperellipsoids and Implications in Optimal Recovery from Inaccurate Data

by S.Foucart and C.Liao

This reproducible was written by S. Foucart and C. Liao in December 2023.

CVXPY [2] is required to execute this reproducible.

In [1]:
import numpy as np
import cvxpy as cp
import scipy as sp

## INTERSECTION OF 2 HYPERELLIPSOIDS

### Setting the problem

In [2]:
# dimension of the ambiant Hilbert space H
N = 20;
# two Hilbert-valued linear maps defined on H
R = np.random.randn(N,N)
S = np.random.randn(N,N)
# dimension of another Hilbert space Z which is the range of quantity of interest
n = 10
# quantity of interest 
Q = np.random.randn(n,N)
# number of observations
m = 7
# observation map
Lam = np.random.randn(m,N)
# construct a basis for ker(Lam)
U_aux,__ = np.linalg.qr(Lam.T, mode='complete')
# the columns of U are an ONB for ker(Lam)
U = U_aux[:,m:]

### Computation of the radius of information (Theorem 1)

In [3]:
a = cp.Variable((1,1), nonneg=True)
b = cp.Variable((1,1), nonneg=True)
objective = cp.Minimize(a+b)
constraints = [U.T@(cp.multiply(a,R.T@R)+cp.multiply(b,S.T@S)-Q.T@Q)@U>>0]
# The symmetric matrices R', S' and Q'defined in Eq.(11) of the paper are computed here as 
# U'*R'*R*U, U'*S'*S*U, and U'*Q'*Q*U, respectively.
sdp1 = cp.Problem(objective,constraints)
sdp1.solve()
a = a.value[0,0]
b = b.value[0,0]
rad = np.sqrt(a+b)
print('For the two-hyperellipsoid-intersection model set,\n'
    'the radius of information of the observation map '
    f'for the estimation of Q is {rad:0.3f}.')

For the two-hyperellipsoid-intersection model set,
the radius of information of the observation map for the estimation of Q is 1.937.


### Verification of the side result (Proposition 6)

In [4]:
# map T_N defined in the proof
TN = U.T@(a*R.T@R+b*S.T@S)@U
TN_aux = np.linalg.inv(sp.linalg.sqrtm(TN))
__,v = np.linalg.eigh(TN_aux@U.T@Q.T@Q@U@TN_aux)
h_aux = U@TN_aux@v[:,-1]
h = h_aux/np.linalg.norm(R@h_aux);
# the norm of Rh and Sh should both be =1
print([np.linalg.norm(R@h), np.linalg.norm(S@h)])
print('')
# the norm of Qh should equal the radius of information
print([np.linalg.norm(Q@h), rad])

[0.9999999999999998, 1.0000421582731926]

[1.9369968870370655, 1.9369539348605072]


## INTERSECTION OF 3 HYPERELLIPSOIDS

following the remark after Proposition 6 at the end of Subsection 2.3

### The norms $\|R_i h_\#\|$ can all be equal to 1 in some situations...

In [5]:
# fix the random seed to guarantee equality of these norms
np.random.seed(1)    # comment out this line to test different instances
# dimension of the ambiant Hilbert space H
N = 20
# three Hilbert-valued linear maps defined on H
R1 = np.random.randn(N,N)
R2 = np.random.randn(N,N)
R3 = np.random.randn(N,N)
# dimension of another Hilbert space Z which is the range of quantity of interest
n = 10
# quantity of interest 
Q = np.random.randn(n,N)
# number of observations
m = 7
# observation map
Lam = np.random.randn(m,N)
# construct a basis for ker(Lam)
U_aux,__ = np.linalg.qr(Lam.T, mode='complete')
# the columns of U are an ONB for ker(Lam)
U = U_aux[:,m:]

# solve an SDP
a1 = cp.Variable((1,1), nonneg=True)
a2 = cp.Variable((1,1), nonneg=True)
a3 = cp.Variable((1,1), nonneg=True)
objective = cp.Minimize(a1+a2+a3)
constraints = [U.T@(cp.multiply(a1,R1.T@R1)+cp.multiply(a2,R2.T@R2) + cp.multiply(a3,R3.T@R3) - Q.T@Q)@U>>0]

sdp2 = cp.Problem(objective,constraints)
sdp2.solve()
a1 = a1.value[0,0]
a2 = a2.value[0,0]
a3 = a3.value[0,0]


# inspired by the side results, we compute norms of R1*h, R2*h, R3*h
TN = U.T@(a1*R1.T@R1+a2*R2.T@R2+a3*R3.T@R3)@U
TN_aux = np.linalg.inv(sp.linalg.sqrtm(TN))
__,v = np.linalg.eigh(TN_aux@U.T@Q.T@Q@U@TN_aux)
h_aux = U@TN_aux@v[:,-1]
h = h_aux/np.linalg.norm(R1@h_aux);
[np.linalg.norm(R1@h), np.linalg.norm(R2@h), np.linalg.norm(R3@h)]

[1.0, 1.0000022599097735, 0.9999999972536573]

Since the norms of $R_1h_\#$, $R_2h_\#$, and $R_3h_\#$ are all equal to 1 in this case, a linear constrained regularization map is optimal.

### ... but the norms $\|R_ih_\#\|$ are not always equal, in particular when the  $R_i$'s are orthogonal projectors

In [6]:
# dimension of the ambiant Hilbert space H
N = 20
# three orthogonal projectors
n1 = 3
V1 = np.random.randn(N,n1)
R1 = np.identity(N) - V1@np.linalg.inv(V1.T@V1)@V1.T
n2 = 5
V2 = np.random.randn(N,n2)
R2 = np.identity(N) - V2@np.linalg.inv(V2.T@V2)@V2.T
n3 = 12
V3 = np.random.randn(N,n3)
R1 = np.identity(N) - V3@np.linalg.inv(V3.T@V3)@V3.T
# dimension of another Hilbert space Z which is the range of quantity of interest
n = 10;
# quantity of interest 
Q = np.random.randn(n,N);
# number of observations
m = 7;
# observation map
Lam = np.random.randn(m,N);
# construct a basis for ker(Lam)
U_aux,__ = np.linalg.qr(Lam.T, mode='complete');
# the columns of U are an ONB for ker(Lam)
U = U_aux[:,m:]; 

# solve an SDP
a1 = cp.Variable((1,1), nonneg=True)
a2 = cp.Variable((1,1), nonneg=True)
a3 = cp.Variable((1,1), nonneg=True)
objective = cp.Minimize(a1+a2+a3)
constraints = [U.T@(cp.multiply(a1,R1.T@R1)+cp.multiply(a2,R2.T@R2) + cp.multiply(a3,R3.T@R3) - Q.T@Q)@U>>0]

sdp2 = cp.Problem(objective,constraints)
sdp2.solve()
a1 = a1.value[0,0]
a2 = a2.value[0,0]
a3 = a3.value[0,0]


# inspired by the side results...
TN = U.T@(a1*R1.T@R1+a2*R2.T@R2+a3*R3.T@R3)@U
TN_aux = np.linalg.inv(sp.linalg.sqrtm(TN))
__,v = np.linalg.eigh(TN_aux@U.T@Q.T@Q@U@TN_aux)
h_aux = U@TN_aux@v[:,-1]
h = h_aux/np.linalg.norm(R1@h_aux);
# usually, the norms of R1*h, R2*h, and R3*h are not equal
[np.linalg.norm(R1@h), np.linalg.norm(R2@h), np.linalg.norm(R3@h)]

[0.9999999999999999, 1.7057597192030172, 2.4623431808309166]

## Two-space problem

following section 3.1

### Setting of the problem

In [7]:
# dimension of the ambiant Hilbert space H
N = 20;
# two orthogonal projectors defined on H
n1 = 5
V = np.random.randn(N,n1)
PV = np.identity(N) - V@np.linalg.inv(V.T@V)@V.T
n2 = 3
W = np.random.randn(N,n2)
PW = np.identity(N) - W@np.linalg.inv(W.T@W)@W.T
# approximability parameters
eps = 0.2; 
eta = 0.8; 
# dimension of another Hilbert space Z which is the range of quantity of interest
n = 10
# quantity of interest 
Q = np.random.randn(n,N)
# number of observations
m = 7
# observation map
Lam = np.random.randn(m,N)
# construct a basis for ker(Lam)
U_aux,__ = np.linalg.qr(Lam.T, mode='complete')
# the columns of U are an ONB for ker(Lam)
U = U_aux[:,m:]

### Computation of the radius of information (Theorem 7)

In [8]:
c = cp.Variable((1,1), nonneg=True)
d = cp.Variable((1,1), nonneg=True)
objective = cp.Minimize(c*eps**2+d*eta**2)
constraints = [U.T@(cp.multiply(c,PV)+cp.multiply(d,PW)-Q.T@Q)@U>>0]
sdp1 = cp.Problem(objective,constraints)
sdp1.solve()
c = c.value[0,0]
d = d.value[0,0]
rad = np.sqrt(c*eps**2+d*eta**2)
print('For the two-space model set,\n'
    'the radius of information of the observation map '
    f'for the estimation of Q is {rad:0.3f}.')

For the two-space model set,
the radius of information of the observation map for the estimation of Q is 2.390.


## L2-INACCURATE DATA

following Section 3.2

Numerical experiments were done in the reproducible files accompanying [3] for $Q=\mathrm{Id}$ and $S=\mathrm{Id}$ and accompanying [4] for an arbitrary $Q$ but still with $S=\mathrm{Id}$. We refer readers to the following links:

Reproducible file for [3]: https://htmlpreview.github.io/?https://github.com/foucart/COR/blob/master/MATLAB/web/ORHilbert_Reg_repro.html

Reproducible file for [4]: https://github.com/liaochunyang/ORofGraphSignals 

## MIXED ACCURATE AND L2-INACCURATE DATA 

following Section 3.3

### setting of the problem

In [9]:
# dimension of the ambiant Hilbert space H
N = 20;
# Hilbert-valued liner map defined on H
R = np.random.randn(N,N)
# dimension of another Hilbert space Z which is the range of quantity of interest
n = 10
# quantity of interest 
Q = np.random.randn(n,N)
# number of observations
m = 7
# observation map
Lam = np.random.randn(m,N)
# two Hilbert-valued linear maps defined on R^m
S1 = np.random.randn(m,m);                       # represents S' in the paper
S2 = np.random.randn(m,m);                       # represents S'' in the paper
# approximability parameter
eps = 0.5;
# uncertainty parameter
eta = 0.3; 
# construct a basis for ker(S1*Lam)
U_aux,__ = np.linalg.qr( (S1@Lam).T, mode='complete')
# the columns of U are an ONB for ker(Lam)
U = U_aux[:,m:]

### Computation of the radius of information (Theorem 9)

In [10]:
c = cp.Variable((1,1), nonneg=True)
d = cp.Variable((1,1), nonneg=True)
objective = cp.Minimize(c*eps**2+d*eta**2)
constraints = [U.T@(cp.multiply(c,R.T@R)+cp.multiply(d,Lam.T@S2.T@S2@Lam)-Q.T@Q)@U>>0]
sdp1 = cp.Problem(objective,constraints)
sdp1.solve()
c = c.value[0,0]
d = d.value[0,0]
rad = np.sqrt(c*eps**2+d*eta**2)
print('For the the mixed accurate and inaccurate model,\n'
    'the radius of information of the observation map '
    f'for the estimation of Q is {rad:0.3f}.')

For the the mixed accurate and inaccurate model,
the radius of information of the observation map for the estimation of Q is 1.909.


## L1-INACCURATE DATA

following Section 4

### Case 1: Small $\eta$

#### Setting the problem

In [11]:
# dimension of the ambiant Hilbert space H
N = 20;
# Hilbert-valued linear map defined on H
R = np.random.randn(N,N)
# approximability parameter
eps = 0.5
# dimension of another Hilbert space Z which is the range of quantity of interest
n = 10
# quantity of interest 
Q = np.random.randn(n,N)
# number of observations
m = 7
# observation map
Lam = np.random.randn(m,N)
# pseudo-inverse of Lam
Lamdag = Lam.T@np.linalg.inv(Lam@Lam.T)
# uncertainty parameter
eta = 0.01

#### Preparations for the calculations to come

We introduce linear maps defined on the extended Hilbert space $H\times\mathbb{R}$ as in Eq.(38) of the paper.

In [12]:
# augmented observation map Gam
Gam = np.column_stack((Lam, np.zeros([m,1])))
# construct a basis for ker(Gam)
U_aux,__ = np.linalg.qr(Gam.T, mode='complete')
# the columns of U are an ONB for ker(Lam)
U = U_aux[:,m:]
# augmented Hilbert-valued linear map S
S = np.zeros([1,N+1])
S[:,-1] = 1/eta
# augmented Hilbert-valued linear maps R^i and Q^i
R_aug = np.zeros((N,N+1,m))
Q_aug = np.zeros((n,N+1,m))
for i in range(m):
    ui = Lamdag[:,i]
    # the i-th element in the tensors R_aug and Q_aug represent R^i and Q^i, respectively
    R_aug[:,:,i] = 1/eps*np.column_stack((R,-R@ui))
    Q_aug[:,:,i] = np.column_stack((Q,-Q@ui))

#### Compute the lower bounds

In [13]:
lb_p = np.zeros((m,1))
# array to store the extremizers a and b
ab = np.zeros((m,2))
for i in range(m):
    Ri = R_aug[:,:,i]
    Qi = Q_aug[:,:,i]
    a = cp.Variable((1,1), nonneg=True)
    b = cp.Variable((1,1), nonneg=True)
    objective = cp.Minimize(a+b)
    constraints = [U.T@(cp.multiply(a,Ri.T@Ri)+cp.multiply(b,S.T@S)-Qi.T@Qi)@U>>0]
    sdp1 = cp.Problem(objective,constraints)
    sdp1.solve()
    a = a.value[0,0]
    b = b.value[0,0]
    lb_p[i] = np.sqrt(a+b)
    ab[i,0], ab[i,1] = a,b

#### Construction of the regularizers

In [14]:
Gamdag = Gam.T@np.linalg.inv(Gam@Gam.T)
Del = np.zeros((n,m,m))
for j in range(m):
    aj = ab[j,0]
    bj = ab[j,1]
    Rj = R_aug[:,:,j]
    Qj = Q_aug[:,:,j]
    aux = Gamdag - U@np.linalg.inv(U.T@(aj*Rj.T@Rj+bj*S.T@S)@U)@U.T@(aj*Rj.T@Rj+bj*S.T@S)@Gamdag
    Del[:,:,j] = Qj@aux   

#### Verification of the sufficient condition

In [15]:
k = np.argmax(lb_p)
Mk = np.zeros((m,1))
for i in range(m):
    Ri = R_aug[:,:,i]
    Qi = Q_aug[:,:,i]
    aux = Qi - Del[:,:,i]@Gam
    a = cp.Variable((1,1), nonneg=True)
    b = cp.Variable((1,1), nonneg=True)
    objective = cp.Minimize(a+b)
    constraints = [cp.multiply(a,Ri.T@Ri) + cp.multiply(b,S.T@S) - aux.T@aux>>0]
    sdp1 = cp.Problem(objective,constraints)
    sdp1.solve()
    a = a.value[0,0]
    b = b.value[0,0]
    Mk[i] = np.sqrt(a+b)
    
# sufficient condition: the maximum of Mk should be equal to lb_p(k)
[ np.max(Mk), lb_p[k,0] ]

[1.205473102819632, 1.2054730967856953]

#### Verification of the side results (proposition 14)

In [16]:
a = cp.Variable((m,1), nonneg=True)
b = cp.Variable((m,1), nonneg=True)
Del_lin = cp.Variable((n,m))        # a linear recovery map treated as a variable in the SDP
gam = cp.Variable((1,1), nonneg=True)
objective = cp.Minimize(gam)
constraints = [a[i]+b[i]<=gam for i in range(m)]
for i in range(m):
    aux = Q_aug[:,:,i] - Del_lin@Gam
    Ri = R_aug[:,:,i]
    Mi1 = cp.hstack( (np.identity(n), aux) ) 
    Mi2 = cp.hstack( (aux.T, cp.multiply(a[i],Ri.T@Ri) + cp.multiply(b[i],S.T@S)) )
    constraints += [cp.vstack((Mi1,Mi2))>>0]
sdp1 = cp.Problem(objective,constraints)
sdp1.solve()
gam = gam.value[0,0]

# the optimal value of this SDP should coincide with the maxima of Mk and lb_p(k),
# confirming that there is a a linear recovery map with minimal global worst-case error
# when eta is small enough
[np.max(Mk), lb_p[k,0], np.sqrt(gam)]

[1.205473102819632, 1.2054730967856953, 1.2054731202339832]

In [17]:
Del_lin = Del_lin.value[:,:]
print('The norm of difference between linear map obtained by regularization \n'
    f'and the linear map obtained as solution of SDP is {np.linalg.norm(Del[:,:,k]-Del_lin):.3f},\n' 
      'illustrating once again the nonuniqueness of optimal recovery maps.')

The norm of difference between linear map obtained by regularization 
and the linear map obtained as solution of SDP is 0.119,
illustrating once again the nonuniqueness of optimal recovery maps.


### Case 2: Large $\eta$

We repeat case 1 and select a large $\eta$.

In [18]:
## Setting the problem 

# dimension of the ambiant Hilbert space H
N = 20;
# Hilbert-valued linear map defined on H
R = np.random.randn(N,N)
# approximability parameter
eps = 0.5
# dimension of another Hilbert space Z which is the range of quantity of interest
n = 10
# quantity of interest 
Q = np.random.randn(n,N)
# number of observations
m = 7
# observation map
Lam = np.random.randn(m,N)
# pseudo-inverse of Lam
Lamdag = Lam.T@np.linalg.inv(Lam@Lam.T)
# uncertainty parameter
eta = 5



## Preparations for the calculations to come

# augmented observation map Gam
Gam = np.column_stack((Lam, np.zeros([m,1])))
# construct a basis for ker(Gam)
U_aux,__ = np.linalg.qr(Gam.T, mode='complete')
# the columns of U are an ONB for ker(Lam)
U = U_aux[:,m:]
# augmented Hilbert-valued linear map S
S = np.zeros([1,N+1])
S[:,-1] = 1/eta
# augmented Hilbert-valued linear maps R^i and Q^i
R_aug = np.zeros((N,N+1,m))
Q_aug = np.zeros((n,N+1,m))
for i in range(m):
    ui = Lamdag[:,i]
    # the i-th elements in the tensors R_aug and Q_aug represent R^i and Q^i, respectively
    R_aug[:,:,i] = 1/eps*np.column_stack((R,-R@ui))
    Q_aug[:,:,i] = np.column_stack((Q,-Q@ui))

    
    
## Compute the lower bounds

lb_p = np.zeros((m,1))
# array to store the extremizers a and b
ab = np.zeros((m,2))
for i in range(m):
    Ri = R_aug[:,:,i]
    Qi = Q_aug[:,:,i]
    a = cp.Variable((1,1), nonneg=True)
    b = cp.Variable((1,1), nonneg=True)
    objective = cp.Minimize(a+b)
    constraints = [U.T@(cp.multiply(a,Ri.T@Ri)+cp.multiply(b,S.T@S)-Qi.T@Qi)@U>>0]
    sdp1 = cp.Problem(objective,constraints)
    sdp1.solve()
    a = a.value[0,0]
    b = b.value[0,0]
    lb_p[i] = np.sqrt(a+b)
    ab[i,0], ab[i,1] = a,b
    
## Construction of the regularizers

Gamdag = Gam.T@np.linalg.inv(Gam@Gam.T)
Del = np.zeros((n,m,m))
for j in range(m):
    aj = ab[j,0]
    bj = ab[j,1]
    Rj = R_aug[:,:,j]
    Qj = Q_aug[:,:,j]
    aux = Gamdag - U@np.linalg.inv(U.T@(aj*Rj.T@Rj+bj*S.T@S)@U)@U.T@(aj*Rj.T@Rj+bj*S.T@S)@Gamdag
    Del[:,:,j] = Qj@aux   

#### Worst-case errors for several linear recovery maps

The computation below is based on Proposition 14

In [19]:
# for the linear map given by regularization
k = np.argmax(lb_p)
a = cp.Variable((m,1), nonneg=True)
b = cp.Variable((m,1), nonneg=True)
gam = cp.Variable((1,1), nonneg=True)
objective = cp.Minimize(gam)
constraints = [a[i]+b[i]<=gam for i in range(m)]
for i in range(m):
    aux = Q_aug[:,:,i] - Del[:,:,k]@Gam
    Ri = R_aug[:,:,i]
    Mi1 = cp.hstack( (np.identity(n), aux) ) 
    Mi2 = cp.hstack( (aux.T, cp.multiply(a[i],Ri.T@Ri) + cp.multiply(b[i],S.T@S)) )
    constraints += [cp.vstack((Mi1,Mi2))>>0]
sdp1 = cp.Problem(objective,constraints)
sdp1.solve()
ub = np.sqrt(gam.value[0,0])

# for a linear recovery map treated as a variable in the SDP
a = cp.Variable((m,1), nonneg=True)
b = cp.Variable((m,1), nonneg=True)
Del_lin = cp.Variable((n,m))
gam = cp.Variable((1,1), nonneg=True)
objective = cp.Minimize(gam)
constraints = [a[i]+b[i]<=gam for i in range(m)]
for i in range(m):
    aux = Q_aug[:,:,i] - Del_lin@Gam
    Ri = R_aug[:,:,i]
    Mi1 = cp.hstack( (np.identity(n), aux) ) 
    Mi2 = cp.hstack( (aux.T, cp.multiply(a[i],Ri.T@Ri) + cp.multiply(b[i],S.T@S)) )
    constraints += [cp.vstack((Mi1,Mi2))>>0]
sdp1 = cp.Problem(objective,constraints)
sdp1.solve()
gam = np.sqrt(gam.value[0,0])

print(f'The computed lower bound on the global worst-case error is {np.max(lb_p):.4f}.\n',)
print(f'The global worst-case error of an optimal linear recovery map is {gam:.4f}.\n')
print(f'The upper bound on the global worst-case error using the earlier regularization map {ub:.4f}.\n')

The computed lower bound on the global worst-case error is 2.4198.

The global worst-case error of an optimal linear recovery map is 8.8989.

The upper bound on the global worst-case error using the earlier regularization map 17.1312.



## References:

[1] S. Foucart and C. Liao, "Radius of Information for Two Intersected Centered Hyperellipsoids and Implications in Optimal Recovery from Inaccurate Data", Preprint.

[2] S. Diamond and S. Boyd, "CVXPY: A Python-embedded modeling language for convex optimization", Journal of Machine Learning Research, 17(83):1–5, 2016.

[3] S. Foucart and C. Liao, "Optimal recovery from inaccurate data in Hilbert spaces: regularize, but what of the parameter?", Constructive Approximation, pages 1–32, 2022. 

[4] S. Foucart, C. Liao and N. Veldt. "On the optimal recovery of graph signals". In 2023 International Conference on Sampling Theory and Applications (SampTA), pages 1–5, 2023. doi: 10.1109/ SampTA59647.2023.10301205. 